In [1]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

# Load dataset
file_path = 'data/bbc-text.csv'
data = pd.read_csv(file_path)

# Display basic information about the dataset
data.info()

# Display the first few rows of the dataset
data.head()

# Tokenize the text
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['text'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(data['text'])

# Pad the sequences
padded_sequences = pad_sequences(sequences, padding='post')

# Encode the labels
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['category'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['label'], test_size=0.2, random_state=42)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [3]:

class LSTMCell:
    def __init__(self, input_dim, hidden_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.Wf = np.random.randn(hidden_dim, hidden_dim + input_dim) * 0.01
        self.bf = np.zeros((hidden_dim, 1))
        self.Wi = np.random.randn(hidden_dim, hidden_dim + input_dim) * 0.01
        self.bi = np.zeros((hidden_dim, 1))
        self.Wc = np.random.randn(hidden_dim, hidden_dim + input_dim) * 0.01
        self.bc = np.zeros((hidden_dim, 1))
        self.Wo = np.random.randn(hidden_dim, hidden_dim + input_dim) * 0.01
        self.bo = np.zeros((hidden_dim, 1))
        
    def forward(self, xt, a_prev, c_prev):
        concat = np.concatenate((a_prev, xt), axis=0)
        
        ft = self.sigmoid(np.dot(self.Wf, concat) + self.bf)
        it = self.sigmoid(np.dot(self.Wi, concat) + self.bi)
        cct = np.tanh(np.dot(self.Wc, concat) + self.bc)
        c_next = ft * c_prev + it * cct
        ot = self.sigmoid(np.dot(self.Wo, concat) + self.bo)
        a_next = ot * np.tanh(c_next)
        
        return a_next, c_next

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))


In [4]:

# Initialize parameters
input_dim = X_train.shape[1]
hidden_dim = 128
output_dim = len(label_encoder.classes_)

# Initialize LSTM cell
lstm_cell = LSTMCell(input_dim, hidden_dim)


In [5]:

def forward_propagation(X, lstm_cell):
    m, T_x = X.shape
    n_a, _ = lstm_cell.Wf.shape
    
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    a_next = np.zeros((n_a, m))
    c_next = np.zeros((n_a, m))
    
    for t in range(T_x):
        xt = X[:, t]
        a_next, c_next = lstm_cell.forward(xt, a_next, c_next)
        a[:, :, t] = a_next
        c[:, :, t] = c_next
    
    return a, c


In [6]:

def compute_loss(y, y_hat):
    m = y.shape[0]
    loss = -1 / m * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    return loss

def train(X, y, lstm_cell, epochs=10, learning_rate=0.01):
    for epoch in range(epochs):
        a, _ = forward_propagation(X, lstm_cell)
        y_hat = a[:, :, -1]
        loss = compute_loss(y, y_hat)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")
        # Backpropagation and parameter update code goes here


In [7]:

# Evaluate the model (Example code, implement evaluation logic as needed)
a, _ = forward_propagation(X_test, lstm_cell)
y_hat = a[:, :, -1]
# Convert predictions to class labels and calculate accuracy


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)